In [32]:
from operator import xor
import random
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import SelectFromModel,mutual_info_classif
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings("ignore")

###############  GENERACIÓN DEL CONJUNTO DE DATOS SINTÉTICOS ##################
dimArray=4
nObservaciones=100
posComb=list(np.ndindex((2,) * dimArray))
array_binario=[]
for i in posComb:
  array_binario.append(list(i))

posComb=array_binario



while len(array_binario)<nObservaciones:
  array_binario.append(random.choice(posComb))

X=[]
Y=[]
for i in array_binario:
    var_4=int(xor(i[1],i[2]))
    X.append(i[:3]+[var_4]+[i[3]])
    Y.append(int(xor(i[0],var_4)))

X=pd.DataFrame(X, columns=['X1','X2','X3','X4','X5'])
Y=pd.DataFrame(Y, columns=['Y'])


############### RANKING + FILTRADO ##################
#------ 1) RANKING + FILTRADO USANDO Chi-cuadrado----------##
k=len(X.columns)
selector=SelectKBest(chi2, k=k)
carSelec=selector.fit_transform(X, Y)

puntuacion=selector.scores_
indices=np.argsort(puntuacion)[::-1]
carOdernadas=X.columns[indices]

print("Ranking de características Chi-cuadrado:")
for i, caracteristica in enumerate(carOdernadas):
    print(f"{i+1}. {caracteristica} ({puntuacion[indices[i]]})")

#------ 2) RANKING + FILTRADO USANDO Arboles de decisión ----------##
k=len(X.columns)
arbol=DecisionTreeClassifier()
sel=SelectFromModel(arbol, max_features=5)
carSelec=sel.fit_transform(X,Y)

inidces=sel.get_support()
carOdernadas=X.columns[indices]

print("Características seleccionadas Arbol:")
for caracteristica in carOdernadas:
    print(caracteristica)

#------ 3) RANKING + FILTRADO USANDO Información Mútua ----------##
k=len(X.columns)
IM=SelectKBest(mutual_info_classif, k=k)
carSelec=IM.fit_transform(X, Y)
puntuacion=IM.scores_
indices=np.argsort(puntuacion)[::-1]
carOdernadas=X.columns[indices]
print("Ranking de características Información Mutua:")
for i, caracteristica in enumerate(carOdernadas):
    print(f"{i+1}. {caracteristica} ({puntuacion[indices[i]]})")



############### ANALISIS COMPONENTES PRINCIPALES ##################
nComponentes=len(X.columns)
pca=PCA(n_components=nComponentes)
X_pca=pca.fit_transform(X)

porVarExp=pca.explained_variance_ratio_
indices=np.argsort(porVarExp)[::-1]

compOrdenados=[f"Componente Principal {i+1}" for i in range(nComponentes)]

carSelec=[X.columns[i] for i in indices]

print("Varianza explicada por orden usando ACP:")
for i, feature in enumerate(carSelec):
    print(f"{i+1}. {feature} (Varianza Explicada = {porVarExp[indices[i]]})")


## MÉTODO ENVOLVENTE (Eliminación Recursiva de Características) RegLogística ###
nCaracteristica=len(X.columns)
estimador=LogisticRegression()

selector=RFE(estimador,n_features_to_select=nCaracteristica,step=1)
selector.fit(X, Y)


puntuaciones=selector.estimator_.coef_[0]
lista=[(nombre,punt) for nombre,punt in zip(X.columns,abs(puntuaciones))]
listaOrdenada=sorted(lista,key=lambda x:x[1],reverse=True)

print("Ranking de características RFE (Regresíon Logística):")
for nombre, punt in listaOrdenada:
    print(f"{nombre}:{punt}")


Ranking de características Chi-cuadrado:
1. X1 (1.875155022736669)
2. X4 (1.4636263725183762)
3. X5 (0.6986357999173214)
4. X2 (0.15006200909466727)
5. X3 (0.06615311709131977)
Características seleccionadas Arbol:
X1
X4
X5
X2
X3
Ranking de características Información Mutua:
1. X2 (0.025570665722636665)
2. X5 (0.020432530165383378)
3. X4 (0.0)
4. X3 (0.0)
5. X1 (0.0)
Varianza explicada por orden usando ACP:
1. X1 (Varianza Explicada = 0.26789574360151247)
2. X2 (Varianza Explicada = 0.2669371695993698)
3. X3 (Varianza Explicada = 0.1930237622967404)
4. X4 (Varianza Explicada = 0.14225519584580978)
5. X5 (Varianza Explicada = 0.1298881286565675)
Ranking de características RFE (Regresíon Logística):
X1:0.8696530230934546
X4:0.8468961960091524
X5:0.4003291764787521
X2:0.3583974383995359
X3:0.1497144725499844
